In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 43.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import spacy
import speech_recognition as sr

In [ ]:
# Load NLP Models
summarizer = pipeline("summarization", model="t5-small")
tone_analyzer = pipeline("text-classification", model="distilbert-base-uncased")
ner_model = spacy.load("en_core_web_sm")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 1. Analyze Tone
def analyze_tone(text):
    result = tone_analyzer(text)
    return result

In [ ]:
# 2. Summarize Grant Requirements
def summarize_requirements(requirements):
    summary = summarizer(requirements, max_length=50, min_length=25, do_sample=False)
    return summary[0]["summary_text"]

In [ ]:
# 3. Extract Named Entities
def extract_ner(text):
    doc = ner_model(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
# 4. Transcribe Voice Notes Using SpeechRecognition
def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    try:
        transcription = recognizer.recognize_google(audio)
        return transcription
    except sr.UnknownValueError:
        return "Could not understand the audio."
    except sr.RequestError:
        return "Speech Recognition service error."

In [ ]:
# Example Usage
if __name__ == "__main__":
    # Input: Grant Draft
    draft = "Our organization is focused on advancing renewable energy solutions in underserved communities."
    tone = analyze_tone(draft)
    print("Tone Analysis:", tone)

    # Input: Grant Requirements
    requirements = """
    The grant application must emphasize innovative solutions for sustainability,
    demonstrate measurable outcomes, and include a detailed budget plan.
    """
    summary = summarize_requirements(requirements)
    print("Summarized Requirements:", summary)

    # Input: Named Entity Recognition
    entities = extract_ner(draft)
    print("Extracted Entities:", entities)

    # Input: Voice Note (replace 'path_to_audio.wav' with actual file path)
    audio_transcription = transcribe_audio("/content/Record.wav")
    print("Transcribed Voice Note:", audio_transcription)

Your max_length is set to 50, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


Tone Analysis: [{'label': 'LABEL_0', 'score': 0.540642499923706}]
Summarized Requirements: grant application must emphasize innovative solutions for sustainability . it must include measurable outcomes and a detailed budget plan .
Extracted Entities: []


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format